In [2]:
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-a7a3  GCE       3                                             RUNNING  us-central1-a


In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import numpy as np
from google.cloud import storage
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  8 10:41 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
spark

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ir_52'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

# Building an Inverted Index

In [9]:
parquetFile = spark.read.parquet(*paths)

# Crate an rdd from the the title and id
doc_text_pairs = parquetFile.select("id", "text").rdd

In [10]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import *

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

In [13]:
def len_tokens(id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  return (id, len(tokens))

# Apply the function to your RDD
len_pairs = doc_text_pairs.map(lambda x: len_tokens(x[0], x[1]))
doc_lengths_dict = len_pairs.collectAsMap()


In [14]:
# Corrected code
name = 'doc_lengths.pkl'
with open(name, 'wb') as f:
    pickle.dump(doc_lengths_dict, f)

bucket = client.bucket(bucket_name)
blob = bucket.blob(f"doc_lengths/{name}")

blob.upload_from_filename(name)


In [ ]:
object_name = 'doc_lengths/doc_lengths.pkl'  # The path to your .pkl file in the bucket

bucket = client.bucket(bucket_name)

# Create a blob object
blob = bucket.blob(object_name)

# Define the local filename to save the downloaded file
local_filename = 'doc_lengths.pkl'

# Download the file from GCP bucket to the local file system
blob.download_to_filename(local_filename)

# Load the dictionary from the .pkl file
with open(local_filename, 'rb') as f:
    doc_lengths_dict = pickle.load(f)

# Calculate Average Length
doc_N = len(doc_lengths_dict)
avg_doc_length = sum(doc_lengths_dict.values())/doc_N
print(doc_N, avg_doc_length)